In [8]:
import agentpy as ap
import matplotlib.pyplot as plt
import numpy as np

# Visualization
import seaborn as sns
import IPython

In [9]:
#Funciones Auxiliares
def iteratorlist(node):

  out = []

  for i in(node):
    out.append(i)

  return out

#Error cuadratico medio
def ecm(pos1, pos2):

  e = 0.5*np.sum(np.power(np.array(pos1) - np.array(pos2), 2))

  return e

In [10]:
class Node(ap.Agent):

    #Constructor
    def setup(self):

        #Reglas de desplazamiento
        self.desp = [[0, 1], [0, -1], [1, 0], [-1, 0]]

        #Reglas de desplazamiento
        self.xlim = 20
        self.ylim = 20

        #Crear ID
        self.id = None

        #Ruta de camino
        self.parents = []

        #Tipo de rama
        self.branch = 0

    #Asignar ID
    def assign_id(self, id):

        #Asignar ID
        self.id = int(id)
        self.parents.append(int(id))

    #Reglas de desplazamiento
    def setup_rules(self, xlim, ylim):

        #Reglas de desplazamiento
        self.xlim = xlim
        self.ylim = ylim

    #Obtener posicion
    def get_position(self, world):

        #Get position
        position = world.positions[self]

        return position

    #Obtener estado siguiente
    def get_next_state(self, world):

        #Obtener posicion
        position = self.get_position(world)

        #Obtener valores de alrededores
        values = []
        for i in range(len(self.desp)):

          #Obtener posicion tras desplazamiento
          pos_x = position[0] + self.desp[i][0]
          pos_y = position[1] + self.desp[i][1]

          #Obtener condicion
          if((pos_x < 0) or (pos_y < 0) or (pos_x > self.xlim - 1) or (pos_y > self.ylim - 1)):
            values.append(-1)
          else:
            #Obtener posicion siguiente
            value = world.agents[(pos_x, pos_y)].condition
            for i in value:
              values.append(i)
              break

        return values

    #Propagar trayectoria
    def propagate(self, world):

      #Obtener posicion del nodo
      position = self.get_position(world)

      #Obtener alrededores
      surroundings = self.get_next_state(world)

      #Propagar camino
      for i in range(len(surroundings)):

        #Posicion del vecindario
        pos = (position[0] + self.desp[i][0],
              position[1] + self.desp[i][1])

        #Espacio libre
        if(surroundings[i] == 0):

          #Cambiar condicion a nodo activo (2)
          world.agents[pos].condition = 2

          #Heredar camino de regreso
          world.agents[pos].parents += self.parents

        #Nodo objetivo
        if(surroundings[i] == 4):

          #Cambiar condicion de nodo objetivo
          world.agents[pos].condition = 5

          #Heredar camino de regreso
          world.agents[pos].parents += self.parents
          break

    #Propagar trayectoria
    def propagate_bidirectional(self, world):

      #Obtener posicion del nodo
      position = self.get_position(world)

      #Obtener alrededores
      surroundings = self.get_next_state(world)

      #Propagar camino
      for i in range(len(surroundings)):

        #Posicion del vecindario
        pos = (position[0] + self.desp[i][0],
              position[1] + self.desp[i][1])

        #Espacio libre
        if(surroundings[i] == 0):

          #Cambiar condicion a nodo activo (2)
          world.agents[pos].condition = 2
          world.agents[pos].branch = self.branch

          #Heredar camino de regreso
          world.agents[pos].parents += self.parents

        #Nodo activo
        if(surroundings[i] == 2):

          #Obtener condicion del vecindario
          current_branch = iteratorlist(world.agents[pos].branch)

          #Verificar si la rama pertenece a otro origen
          if(current_branch[0] != self.branch):
            #Cambiar condicion de nodo objetivo
            world.agents[pos].condition = 5
            world.agents[pos].parents += self.parents
            break

    #Propagar hacia el mejor
    def propagate_best(self, world, goal):

      #Encontrar posiciones del nodo actual y objetivo
      pos = self.get_position(world)
      pos_goal = goal.get_position(world)

      #Obtener alrededores del nodo
      surroundings = self.get_next_state(world)

      #Buscar espacio mas cercano al objetivo
      best_pos = None
      min_cost = 1000
      for i in range(len(surroundings)):

        #Obtener posicion del espacio
        pos_space = (pos[0] + self.desp[i][0],
                    pos[1] + self.desp[i][1])

        #Verificar espacio libre
        if(surroundings[i] == 0):

          #Calcular costo
          cost = ecm(pos_space, pos_goal)

          #Guardar mejor opcion
          if(cost < min_cost):
            best_pos = pos_space
            min_cost = cost

        #Condicion de quiebre
        if(surroundings[i] == 4):

          #Cambiar condicion de nodo objetivo
          world.agents[pos].condition = 5

          #Heredar camino de vuelta
          world.agents[pos].parents+= self.parents
          break

      #Crecer nodo hacia mejor posicion
      if(best_pos != None):

        #Cambiar condicion de espacio a nodo activo
        world.agents[best_pos].condition = 2
        world.agents[best_pos].branch = self.branch

        #Heredar camino
        world.agents[best_pos].parents += self.parents
      #Desactivar nodo si no encontro ningun camino
      else:
        self.condition = 6   #Condicion de nodo inactivo o muerto


    #Propagacion random
    def propagate_random(self, world):

      #Encontrar posiciones del nodo actual y objetivo
      pos = self.get_position(world)

      #Obtener alrededores del nodo
      surroundings = self.get_next_state(world)

      #Obtener espacios libres
      free_spaces = []
      for i in range(len(surroundings)):

        #Espacio libre
        if(surroundings[i] == 0):
          free_spaces.append(i)

        #Condicion de quiebre:
        if(surroundings[i] == 4):

          #Obtener posicion del nodo
          pos = (pos[0] + self.desp[i][0],
                 pos[1] + self.desp[i][1])

          #Cambiar condicion del nodo a finalizado
          world.agents[pos].condition = 5

          #Heredar camino de regreso
          world.agents[pos].parents += self.parents
          break

      #Elegir posicion libre de forma aleatoria
      if(len(free_spaces) > 0):

        #Elegir direccion
        index = free_spaces[np.random.randint(len(free_spaces))]

        #Obtener posicion del nodo
        pos = (pos[0] + self.desp[index][0],
               pos[1] + self.desp[index][1])

        #Cambiar condicion de nodo destino
        world.agents[pos].condition = 2
        world.agents[pos].branch = self.branch

        #Heredar camino de regreso
        world.agents[pos].parents += self.parents
      #Desactivar nodo en caso de que no se pueda expandir
      else:
        self.condition = 6


In [11]:
class RandomMap(ap.Model):

    #Metodo de configuracion
    def setup(self):

      #Obtener tipo de busqueda
      self.mode = self.p['mode']
      self.algorithm = self.p['algorithm']
      self.gains = self.p['gains']

      #Crear agentes del piso
      n_agents = int(self.p['dim']**2)
      self.floor_agents = ap.AgentList(self, n_agents, Node)

      #Crear espacio
      self.floor = ap.Grid(self, [self.p['dim']]*2, track_empty=True)
      self.floor.add_agents(self.floor_agents)

      #Definir espacio libre
      self.floor_agents.condition = 0

      #Definir obstaculos
      obstacle_index = np.random.randint(len(self.floor_agents), size = int(self.p['obs_den']*n_agents))
      for i in range(obstacle_index.shape[0]):
        self.floor_agents[obstacle_index[i]].condition = 1

      #Crear nodo raiz
      self.start_index = np.random.randint(len(self.floor_agents))
      self.floor_agents[self.start_index].condition = 2

      #Crear nodo objetivo
      self.goal_index = np.random.randint(len(self.floor_agents))
      self.floor_agents[self.goal_index].condition = 4

      #Asignar ID unico a nodos
      for i in range(len(self.floor_agents)):
        self.floor_agents[i].assign_id(i)

      #Asignar ramas
      self.floor_agents[self.start_index].branch = 0
      self.floor_agents[self.goal_index].branch = 1

    #Iteracion
    def step(self):

      #Buscar nodos activos
      active_nodes = []

      #1- Busqueda por anchura
      if(self.algorithm == 0):
        for agent in self.floor_agents:

          #Verificar condicion
          if(agent.condition == 2):
            active_nodes.append(agent)
          if(self.mode == 1):
            if(agent.condition == 4):
                active_nodes.append(agent)

      #2- Busqueda por el mejor
      if(self.algorithm == 1 or self.algorithm == 3):

        #Inicializar variables
        min_cost = 1000
        best_agent = None

        #Obtener posicion del objetivo
        pos_goal = self.floor_agents[self.goal_index].get_position(self.floor)

        #Buscar mejor nodo
        for agent in self.floor_agents:

          #Nodos activos
          if(agent.condition == 2):

            #Obtener posicion del agente
            pos = agent.get_position(self.floor)

            #Calcular costo
            cost = ecm(pos, pos_goal)

            #Encontrar mejor
            if(cost < min_cost):
              best_agent = agent
              min_cost = cost

        #Elegir nodo a expandir
        active_nodes.append(best_agent)

      #3- Busqueda A*
      if(self.algorithm == 2):

        #Inicializar variables
        min_cost = 1000
        best_agent = None

        #Obtener posicion del objetivo
        pos_goal = self.floor_agents[self.goal_index].get_position(self.floor)

        #Calcular posicion del origen
        pos_orig = self.floor_agents[self.start_index].get_position(self.floor)

        #Buscar mejor nodo
        for agent in self.floor_agents:

          #Nodos activos
          if(agent.condition == 2):

            #Obtener posicion del agente
            pos = agent.get_position(self.floor)

            #Calcular costo
            cost_goal = ecm(pos, pos_goal) #Costo al objetivo
            cost_orig = ecm(pos, pos_orig) #Costo desde el origen

            #Calcular costo total
            cost = self.gains[0]*cost_goal + self.gains[1]*cost_orig

            #Encontrar mejor
            if(cost < min_cost):
              best_agent = agent
              min_cost = cost

        #Elegir nodo a expandir
        active_nodes.append(best_agent)


      #Ejecutar metodo de busqueda
      #a) Busqueda Unidireccional
      if(self.mode == 0):

        #Propagar nodos activos
        for agent in active_nodes:

          #1- Busqueda por anchura
          if(self.algorithm == 0):

            #Crecer ramas
            agent.propagate(self.floor)

          #2- Busqueda primero el mejor
          if(self.algorithm == 1):

            #Crecer ramas hacia el mejor
            agent.propagate_best(self.floor, self.floor_agents[self.goal_index])

          #3- Busqueda A*
          if(self.algorithm == 2):

            #Crecer ramas hacia el mejor
            agent.propagate_best(self.floor, self.floor_agents[self.goal_index])

          #4- Busqueda aleatoria
          if(self.algorithm == 3):

            #Crecer ramas aleatoriamente
            agent.propagate_random(self.floor)

      #b) Busqueda por anchura bidireccional
      if(self.mode == 1):

        #Propagar nodos activos
        for agent in active_nodes:

          #1- Busqueda por anchura
          if(self.algorithm == 0):
            #Crecer ramas
            agent.propagate_bidirectional(self.floor)

          #2- Busqueda primero el mejor
          if(self.algorithm == 1):
            pass

          #3- Busqueda A*
          if(self.algorithm == 2):
            pass

          #4- Busqueda aleatoria
          if(self.algorithm == 3):
            pass

      #Buscar si algoritmo se completo
      for agent in self.floor_agents:

        #Verificar condicion 5
        if(agent.condition == 5):

          #Recuperar camino de vuelta
          self.draw_path(agent)

          #Detener ejecucion
          self.stop()

    #Marcar camino
    def draw_path(self, node):

      #Obtener camino del objetivo
      path = []
      for parent in node.parents:
        path.append(parent)

      #Cambiar estado de los nodos
      for j in range(len(path)):
        for agent in self.floor_agents:
          if(agent.id == path[j]):
            agent.condition = 3

    #Finalizar simulacion
    def end(self):

        print("Camino encontrado!")
        self.floor_agents[self.goal_index].condition == 3

In [12]:
# Inicializar parametros
parameters = {
    'obs_den': 0.3, # Percentage of grid covered by obstacles
    'dim': 20, # Height and length of the grid
    'mode': 0,  #Modo: 0-Unidireccional, 1-Bidireccional
    'algorithm': 3, #Algoritmo: 0-Anchura, 1-Mejor, 2-A*, 3-Random
    'gains': [0.9, 0.1]  #Goal gain, origin gain
}

In [13]:
#Crear instancia
model = RandomMap(parameters)

In [14]:
# Create single-run animation with custom colors
def rgb2hex(r, g, b):

  var = '#%02x%02x%02x' % (r, g, b)

  return var

# Create single-run animation with custom colors
def animation_plot(model, ax):
    attr_grid = model.floor.attr_grid('condition')
    color_dict = {0:rgb2hex(255, 255, 255), 1:rgb2hex(0, 0, 0), 2:rgb2hex(128, 128, 128), 3:rgb2hex(0, 255, 0), 4:rgb2hex(255, 0, 0), 5:rgb2hex(0, 0, 255), 6:rgb2hex(0, 255, 255), None:rgb2hex(255, 255, 255)}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)

fig, ax = plt.subplots()
animation = ap.animate(model, fig, ax, animation_plot, steps = 100)
IPython.display.HTML(animation.to_jshtml(fps=15))